In [1]:
import numpy as np
import pandas as pd
from importlib import reload

import munch
import datetime

import shared
import config
import provider_yfinance as provider

reload(shared)
reload(config)
reload(provider)

cfg = config.get_config('^GDAXI')

# overwrite download_end_dt: use cached data
#config.overwrite_end_dt(cfg, '2019-12-19')
#config.save_config(cfg)

config> created config from file: './config.json'
config> config
        - base:
            - config_file_path: /mnt/c/notebooks/sandbox/config.json
        - datasets:
            - stocks: 30
            - benchmarks: 69
        - prepare:
            - data_start_dt: 2018-02-16
            - data_end_dt: 2020-01-16
            - cache_dir: /mnt/c/notebooks/sandbox/cache/20200116/
        - train:            
            - window_trading_days: [3, 5, 21, 35, 50]
            - lag_trading_days: [1, 2, 3, 4, 5]
            - label_max_high_weight: 3.0
            - label_max_close_weight: 1.0
            - settings: 12
        - model:
            - max_samples: 40
            - batch_size: 32
            - learning_rate: 0.1
            - learning_rate_decay: 0.75
            - lstm_hidden_size: 256
            - early_stopping_patience: 10
            - validation_monitor: val_mean_squared_error
            - max_epochs: 1000
            - base_dir: /mnt/c/notebooks/sandbox/model/20

In [2]:
%%time 
# 344 ms
cfg_stocks, data_stocks = provider.load_stocks(cfg)
cfg_benchmarks, data_benchmarks = provider.load_benchmarks(cfg)

CPU times: user 250 ms, sys: 109 ms, total: 359 ms
Wall time: 364 ms


In [3]:
data_benchmarks.tickers['^GDAXI'].history.head()

,open,high,low,close,volume,dividends,stock_splits
date,,,,,,,
1987-12-30,1005.19,1005.19,1005.19,1005.19,0,0,0
1988-01-04,956.49,956.49,956.49,956.49,0,0,0
1988-01-05,996.10,996.10,996.10,996.10,0,0,0
1988-01-06,1006.01,1006.01,1006.01,1006.01,0,0,0
1988-01-07,1014.47,1014.47,1014.47,1014.47,0,0,0


In [4]:
%%time 
# total: 2.55 s
prep_stocks = provider.prepare_stocks(cfg, data_stocks)
prep_benchmarks = provider.prepare_benchmarks(cfg, data_benchmarks)

shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200116//stocks_prep.pkl'
shared> prepare stock '1COV.DE' from '2018-02-16' (avail: '2015-10-06') to '2020-01-16'
shared> prepare stock 'ADS.DE' from '2018-02-16' (avail: '1998-06-24') to '2020-01-16'
shared> prepare stock 'ALV.DE' from '2018-02-16' (avail: '1996-12-16') to '2020-01-16'
shared> prepare stock 'BAS.DE' from '2018-02-16' (avail: '1996-12-16') to '2020-01-16'
shared> prepare stock 'BAYN.DE' from '2018-02-16' (avail: '1996-12-16') to '2020-01-16'
shared> prepare stock 'BEI.DE' from '2018-02-16' (avail: '1998-07-22') to '2020-01-16'
shared> prepare stock 'BMW.DE' from '2018-02-16' (avail: '1996-11-08') to '2020-01-16'
shared> prepare stock 'CON.DE' from '2018-02-16' (avail: '1996-12-16') to '2020-01-16'
shared> prepare stock 'DAI.DE' from '2018-02-16' (avail: '1996-10-30') to '2020-01-16'
shared> prepare stock 'DB1.DE' from '2018-02-16' (avail: '2001-02-05') to '2020-01-16'
shared> prepare stock 'DBK.DE' fr

In [5]:
prep_benchmarks['^GDAXI'].head()

,open,high,low,close,volume,diff_prev,diff_oc,diff_hl,rolling_3d,rolling_5d,rolling_21d,rolling_35d,rolling_50d,lag_1d,lag_2d,lag_3d,lag_4d,lag_5d
date,,,,,,,,,,,,,,,,,,
2018-02-16,12408.48,12484.12,12368.71,12451.96,104230500,NaN,43.48,115.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-19,12495.07,12505.49,12361.86,12385.60,66917000,43.11,-109.47,143.63,NaN,NaN,NaN,NaN,NaN,-0.005344,NaN,NaN,NaN,NaN
2018-02-20,12403.63,12496.10,12333.79,12487.90,87127600,18.03,84.27,162.31,12441.820000,NaN,NaN,NaN,NaN,0.008226,0.002882,NaN,NaN,NaN
2018-02-21,12457.22,12474.37,12372.26,12470.49,83382600,-30.68,13.27,102.11,12447.996667,NaN,NaN,NaN,NaN,-0.001395,0.006831,0.001487,NaN,NaN
2018-02-22,12358.82,12497.71,12283.70,12461.91,104173000,-111.67,103.09,214.01,12473.433333,12451.572,NaN,NaN,NaN,-0.000688,-0.002083,0.006142,0.000799,NaN


In [6]:
%%time
# total: 1.2 s
enc_stocks = provider.encode_stocks(cfg, prep_stocks)
enc_benchmarks = provider.encode_benchmarks(cfg, prep_benchmarks, prep_stocks)

shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200116//stocks_enc.pkl'
shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200116//benchmarks_enc.pkl'
CPU times: user 1.17 s, sys: 62.5 ms, total: 1.23 s
Wall time: 1.3 s


In [7]:
enc_benchmarks['^GDAXI'].head()

,open,high,low,close,volume,diff_prev,diff_oc,diff_hl,rolling_3d,rolling_5d,rolling_21d,rolling_35d,rolling_50d,lag_1d,lag_2d,lag_3d,lag_4d,lag_5d,scaled_volume
date,,,,,,,,,,,,,,,,,,,
2018-02-16,12408.48,12484.12,12368.71,12451.96,104230500.0,43.11,43.48,115.41,12441.820000,12451.572,12336.408095,12241.387143,12318.2388,-0.005344,0.002882,0.001487,0.000799,0.002553,0.260469
2018-02-19,12495.07,12505.49,12361.86,12385.60,66917000.0,43.11,-109.47,143.63,12441.820000,12451.572,12336.408095,12241.387143,12318.2388,-0.005344,0.002882,0.001487,0.000799,0.002553,0.167223
2018-02-20,12403.63,12496.10,12333.79,12487.90,87127600.0,18.03,84.27,162.31,12441.820000,12451.572,12336.408095,12241.387143,12318.2388,0.008226,0.002882,0.001487,0.000799,0.002553,0.217729
2018-02-21,12457.22,12474.37,12372.26,12470.49,83382600.0,-30.68,13.27,102.11,12447.996667,12451.572,12336.408095,12241.387143,12318.2388,-0.001395,0.006831,0.001487,0.000799,0.002553,0.208370
2018-02-22,12358.82,12497.71,12283.70,12461.91,104173000.0,-111.67,103.09,214.01,12473.433333,12451.572,12336.408095,12241.387143,12318.2388,-0.000688,-0.002083,0.006142,0.000799,0.002553,0.260325


In [8]:
%%time 
# total: 20min - 50min

for submodel_settings in cfg.train.settings:
    print(f"sm-{submodel_settings.id}> preparing submodel data ...")
    model_data = provider.prepare_submodel_data(cfg, submodel_settings, enc_stocks, enc_benchmarks)
    # update num_features setting (informational)
    submodel_settings.num_features = len(model_data.X[0][0][0][0])
config.save_config(cfg)

sm-lookback_3-label_1> preparing submodel data ...
shared> failed to load pickle file: '/mnt/c/notebooks/sandbox/cache/20200116/lookback_3-label_1/submodel_data.pkl'
sm-lookback_3-label_1> generating relative benchmarks data ...
sm-lookback_3-label_1> preparing stock: '1COV.DE' ...


Using TensorFlow backend.


sm-lookback_3-label_1> preparing stock: 'ADS.DE' ...
sm-lookback_3-label_1> preparing stock: 'ALV.DE' ...
sm-lookback_3-label_1> preparing stock: 'BAS.DE' ...
sm-lookback_3-label_1> preparing stock: 'BAYN.DE' ...
sm-lookback_3-label_1> preparing stock: 'BEI.DE' ...
sm-lookback_3-label_1> preparing stock: 'BMW.DE' ...
sm-lookback_3-label_1> preparing stock: 'CON.DE' ...
sm-lookback_3-label_1> preparing stock: 'DAI.DE' ...
sm-lookback_3-label_1> preparing stock: 'DB1.DE' ...
sm-lookback_3-label_1> preparing stock: 'DBK.DE' ...
sm-lookback_3-label_1> preparing stock: 'DPW.DE' ...
sm-lookback_3-label_1> preparing stock: 'DTE.DE' ...
sm-lookback_3-label_1> preparing stock: 'EOAN.DE' ...
WARN seq-0> no previous year data available!
sm-lookback_3-label_1> preparing stock: 'FME.DE' ...
sm-lookback_3-label_1> preparing stock: 'FRE.DE' ...
WARN seq-1> no further previous year data available!
WARN seq-2> no further previous year data available!
WARN seq-3> no further previous year data available!

In [9]:
# rel_benchmarks_data = provider.generate_relative_benchmarks_data(cfg, cfg.train.settings[0], enc_benchmarks)

In [10]:
submodel_settings = cfg.train.settings[11]
model_data = provider.prepare_submodel_data(cfg, submodel_settings)
model_data

,ticker,date,X,y
0,1COV.DE,2019-01-10,[[[[69.8875351 70.64493561 68.30387878 ... -2...,6.776452
1,1COV.DE,2019-01-11,[[[[72.74633026 72.8628006 70.02096558 ... -2...,8.368274
2,1COV.DE,2019-01-14,[[[[70.68486786 72.90605927 70.56918335 ... -2...,7.641138
3,1COV.DE,2019-01-15,[[[[72.46813202 75.04055786 72.46813202 ... 1...,7.815759
4,1COV.DE,2019-01-16,[[[[69.77848053 72.03887939 68.21880341 ... 2...,5.159358
...,...,...,...,...
3833,WDI.DE,2020-01-07,[[[[26.06168938 27.62628555 23.42422867 ... 0...,16.942333
3834,WDI.DE,2020-01-07,[[[[26.06168938 27.62628555 23.42422867 ... 0...,16.942333
3835,WDI.DE,2020-01-07,[[[[26.06168938 27.62628555 23.42422867 ... 0...,16.942333
3836,WDI.DE,2020-01-07,[[[[26.06168938 27.62628555 23.42422867 ... 0...,16.942333


In [11]:
ticker_name = '1COV.DE'
ticker_data = enc_stocks[ticker_name] 
samples_iter = provider.generate_samples_iterator(cfg, submodel_settings, ticker_data)
samples_iter[:10]

[Munch({'seq_nr': 1, 'lookback_start_date': Timestamp('2018-08-15 00:00:00'), 'lookback_end_date': Timestamp('2019-01-10 00:00:00'), 'label_start_date': Timestamp('2019-01-11 00:00:00'), 'label_end_date': Timestamp('2019-01-21 00:00:00')}),
 Munch({'seq_nr': 2, 'lookback_start_date': Timestamp('2018-08-16 00:00:00'), 'lookback_end_date': Timestamp('2019-01-11 00:00:00'), 'label_start_date': Timestamp('2019-01-14 00:00:00'), 'label_end_date': Timestamp('2019-01-22 00:00:00')}),
 Munch({'seq_nr': 3, 'lookback_start_date': Timestamp('2018-08-17 00:00:00'), 'lookback_end_date': Timestamp('2019-01-14 00:00:00'), 'label_start_date': Timestamp('2019-01-15 00:00:00'), 'label_end_date': Timestamp('2019-01-23 00:00:00')}),
 Munch({'seq_nr': 4, 'lookback_start_date': Timestamp('2018-08-20 00:00:00'), 'lookback_end_date': Timestamp('2019-01-15 00:00:00'), 'label_start_date': Timestamp('2019-01-16 00:00:00'), 'label_end_date': Timestamp('2019-01-24 00:00:00')}),
 Munch({'seq_nr': 5, 'lookback_start

In [12]:
model_data

,ticker,date,X,y
0,1COV.DE,2019-01-10,[[[[69.8875351 70.64493561 68.30387878 ... -2...,6.776452
1,1COV.DE,2019-01-11,[[[[72.74633026 72.8628006 70.02096558 ... -2...,8.368274
2,1COV.DE,2019-01-14,[[[[70.68486786 72.90605927 70.56918335 ... -2...,7.641138
3,1COV.DE,2019-01-15,[[[[72.46813202 75.04055786 72.46813202 ... 1...,7.815759
4,1COV.DE,2019-01-16,[[[[69.77848053 72.03887939 68.21880341 ... 2...,5.159358
...,...,...,...,...
3833,WDI.DE,2020-01-07,[[[[26.06168938 27.62628555 23.42422867 ... 0...,16.942333
3834,WDI.DE,2020-01-07,[[[[26.06168938 27.62628555 23.42422867 ... 0...,16.942333
3835,WDI.DE,2020-01-07,[[[[26.06168938 27.62628555 23.42422867 ... 0...,16.942333
3836,WDI.DE,2020-01-07,[[[[26.06168938 27.62628555 23.42422867 ... 0...,16.942333


In [13]:
data_stocks.tickers['1COV.DE'].history.tail(10).close

date
2020-01-03    41.90
2020-01-06    39.89
2020-01-07    40.13
2020-01-08    40.83
2020-01-09    41.17
2020-01-10    40.64
2020-01-13    40.71
2020-01-14    40.50
2020-01-15    40.12
2020-01-16    40.59
Name: close, dtype: float64

In [14]:
submodel_settings

Munch({'ensemble_weight': 1.0, 'float_precision': 100.0, 'id': 'lookback_100-label_7', 'label_days': 7, 'lookback_days': 100, 'max_samples': 26, 'prev_year_samples_after': 12, 'prev_year_samples_before': 5, 'sample_manifolds': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 5, 6, 7], 'num_features': 1319})

In [15]:
si = samples_iter[0]
base_price = ticker_data.loc[si.lookback_end_date].close
label_data = ticker_data.iloc[(ticker_data.index >= si.label_start_date) & (ticker_data.index <= si.label_end_date)]    
lookback_data = ticker_data.iloc[(ticker_data.index >= si.lookback_start_date) & (ticker_data.index <= si.lookback_end_date)]

y = ((label_data.high.max() * cfg.train.label_max_high_weight) + (label_data.close.max() * cfg.train.label_max_close_weight)) / (cfg.train.label_max_close_weight + cfg.train.label_max_high_weight)
y = (y - base_price) / base_price * submodel_settings.float_precision
y    

6.7764516869405576

In [16]:
((label_data.high.max() * cfg.train.label_max_high_weight) + (label_data.close.max() * cfg.train.label_max_close_weight)) / (cfg.train.label_max_close_weight + cfg.train.label_max_high_weight)

46.5225

In [17]:
label_data.close.max()

46.38

In [18]:
base_price

43.57